<a href="https://colab.research.google.com/github/hyunjung-kim0524/DL_Study/blob/main/20240507_3_3_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


수치형 피처 정규화하기

In [1]:
import pandas as pd

file_url = "https://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
heart_df = pd.read_csv(file_url)

#heart_df.head()
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    object 
 13  target    303 non-null    int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 33.3+ KB


In [2]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization

cols = ['age','trestbps','chol','thalach','oldpeak','slope']
df = heart_df[cols].to_numpy()

layer = Normalization()  # Normalization() : 정규화 함수
layer.adapt(df)   # 데이터 집랍 분석
layer_df = layer(df)

print("features mean : %.1f" %(layer_df.numpy().mean()))
print("features std : %.1f" %(layer_df.numpy().std()))

features mean : -0.0
features std : 1.0


In [3]:
heart_df[cols]
pd.DataFrame(layer_df.numpy(), columns=cols)

,age,trestbps,chol,thalach,oldpeak,slope
0,0.933839,0.745781,-0.260087,0.034807,1.068045,2.284959
1,1.378211,1.592329,0.757388,-1.780617,0.380229,0.663548
2,1.378211,-0.665132,-0.336877,-0.872905,1.325976,0.663548
3,-1.954575,-0.100767,0.066273,1.634109,2.099770,2.284959
4,-1.510204,-0.100767,-0.816818,0.985743,0.294252,-0.957863
...,...,...,...,...,...,...
298,-0.288182,-0.778005,-1.162375,1.763782,-0.909428,0.663548
299,-1.288018,0.012106,1.813258,-0.570334,1.669885,0.663548
300,1.156025,0.181416,0.143064,-0.959353,1.497931,0.663548
301,-0.732554,-0.100767,0.181459,0.034807,-0.909428,-0.957863


문자 범주형 피처 인코딩

In [4]:
heart_df['thal'].value_counts()

thal
normal        168
reversible    115
fixed          18
1               1
2               1
Name: count, dtype: int64

In [5]:
heart_df['thal'][:10]

0         fixed
1        normal
2    reversible
3        normal
4        normal
5        normal
6        normal
7        normal
8    reversible
9    reversible
Name: thal, dtype: object

In [6]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding

df = heart_df['thal'].to_numpy()

layer1 = StringLookup()  # 문자값을 정수 인덱스로 매핑
layer1.adapt(df)

layer2 = CategoryEncoding(num_tokens=6, output_mode="one_hot")  # 정수 인덱스로 매핑된 값을 정수 범주형 특성을 encoding
layer2_df = layer2(layer1(df))

print(type(layer2_df))
print(layer2_df[:10])

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]], shape=(10, 6), dtype=float32)


정수 범주형 피처 인코딩하기

In [9]:
heart_df['fbs'].value_counts()
heart_df['fbs'][:10]

0    1
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    1
Name: fbs, dtype: int64

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

df = heart_df['fbs'].to_numpy()

layer1 = preprocessing.IntegerLookup()  # 정수 범주형 데이터 값을 정수 인덱스로 매핑
layer1.adapt(df)

layer2 = preprocessing.CategoryEncoding(num_tokens=3, output_mode="one_hot")
layer2_df = layer2(layer1(df))

print(type(layer2_df))
print(layer2_df[:10])

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]], shape=(10, 3), dtype=float32)


수치형 피처 인코딩

In [18]:
heart_df['age'].value_counts().count()
heart_df['age'][:10]

0    63
1    67
2    67
3    37
4    41
5    56
6    62
7    57
8    63
9    53
Name: age, dtype: int64

In [19]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

df = heart_df['age'].to_numpy()

layer1 = preprocessing.Hashing(num_bins=16) # 여러개의 피처수를 특성을 유지하는 피처수로 줄여줌

layer2 = preprocessing.CategoryEncoding(num_tokens=16, output_mode = "one_hot")
layer2_df = layer2(layer1(df))


print(layer2_df[:10])

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 16), dtype=float32)


토큰 인덱스로 자연어 인코딩하기

In [31]:
from tensorflow.keras.preprocessing import text_dataset_from_directory

!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

aclImdb_ds = text_dataset_from_directory(
    directory = "/content/aclImdb/train",
    labels = "inferred",
    batch_size = 32,
    seed = 0
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5138k      0  0:00:15  0:00:15 --:--:-- 8946k
Found 75000 files belonging to 3 classes.


In [35]:
for text, label in aclImdb_ds.take(1):
  for i in range(1):
    print(text.numpy()[i])
    print(label.numpy()[i])


b"Super Dimensional Fortress Macross: Do You Remember Love? Ah, what a name and what a movie. This is what classic Japanese Animation is all about. Just the name itself conjures up nostalgic memories of yesterday, running home from fourth grade to catch the latest episode of Robotech, to attending my first Anime convention and realizing the true power of the unedited series as well as this incredible film which retells the story. The music, the characters, the animation and the story are the very things that attracted me to Japanese animation in the first place. And still, even after all of these years, SDFM:DYRL not only holds up, but is still a force to be reckoned with.<br /><br />This is due, in no small part, to the creative talent and storytelling capabilities of Shoji Kawamori. Although his filmography may be small, it is an example of quality over quantity. Kawamori's Macross series made giant transforming robots, singing pop-idol young starlets, heroic floppy haired boys, and 

In [36]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

layer1 = preprocessing.TextVectorization()  # 자연어를 Enbedding, Dense층에서 읽을 수 있도록 인코딩 해줌
layer1.adapt(aclImdb_ds.map(lambda text, label: text))
layer1_df = layer1(['Playing from Time'])

print(type(layer1_df))
print(layer1_df)

<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([[382  36  62]], shape=(1, 3), dtype=int64)


In [39]:
import numpy as np

vocab = np.array(layer1.get_vocabulary())
vocab[10:40]

array(['this', 'i', 'that', 'br', 'was', 'as', 'with', 'for', 'movie',
       'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'be',
       'he', 'one', 'its', 'at', 'all', 'by', 'an', 'they', 'from', 'who',
       'so', 'like'], dtype='<U77')